<a href="https://colab.research.google.com/github/charbells/1864093_ProcesamientoDeDatos/blob/main/1864093_Tarea6Reporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Universidad Autónoma de Nuevo León
# Facultad de Ciencias Físico Matemáticas
# Maestría en Ciencia de Datos

## Procesamiento y clasificación de datos

###### Erick Charbel Lopez Salazar 1864093


# Clasificación de Sonidos Urbanos con Redes Neuronales Convolucionales y Procesamiento de Señales

#Introducción
Las Redes Neuronales Convolucionales (CNN por sus siglas en inglés) son un tipo de arquitectura de deep learning especialmente efectiva para procesar datos con estructura espacial o temporal, como imágenes o señales de audio. En este proyecto, utilizamos una CNN 1D (para datos secuenciales) combinada con técnicas de procesamiento de señales para clasificar sonidos urbanos.

¿Por qué usar CNNs para audio?

Extracción automática de características: A diferencia de métodos tradicionales que requieren ingeniería manual de features (como MFCCs estáticos), las CNN aprenden patrones jerárquicos directamente de los datos.

Captura de dependencias temporales: Las capas convolucionales detectan patrones locales en ventanas de tiempo (ej: un ladrido de perro tiene firmas espectrales distintivas en ciertas frecuencias).

Robustez a variaciones: Invariantes a pequeñas deformaciones en la señal (útil para sonidos con ruido ambiental).

El objetivo de este proyecto fue desarrollar un modelo de aprendizaje automático capaz de clasificar sonidos urbanos del dataset UrbanSound8K, que contiene 10 clases como "air_conditioner", "car_horn", "gun_shot", entre otros.

Se implementó una CNN con transformada wavelet y MFCCs, combinando técnicas de procesamiento de señales y deep learning para lograr una clasificación robusta.

#Datos
Dataset utilizado:

*   UrbanSound8K de hugging face (8,732 audios, 10 clases diferentes).
*   Distribución desigual de muestras (ej: "car_horn" tiene 86 muestras vs. "street_music" con 200).

Normalización y Padding:

*   Escalado de amplitud: Normalización a [-1, 1].

*   Ajuste de duración: Todos los audios se truncaron o rellenaron (padding) a 4 segundos (para uniformidad).

Balanceo de Clases con SMOTE:

*   Problema: Clases minoritarias (gun_shot, car_horn) afectan el aprendizaje.

*   Solución: Se aplicó SMOTE (Synthetic Minority Over-sampling Technique) para generar muestras sintéticas y equilibrar el dataset.

A continuación se muestra la imagen de ejemplo de un sonido:
![imagen 1](https://drive.google.com/uc?id=1s_jkjZgELIoFBRb130qjClEclEULd4gy)

#Metodología
Extracción de características
Wavelet (Db4):

*   Descomposición en 5 niveles.

*   Estadísticas por nivel (media, desviación, percentiles).

MFCCs + Deltas:

*   13 coeficientes + derivadas temporales.

Otras características espectrales:

*   Centroid, bandwidth, zero-crossing rate.

Modelo CNN:

*   3 capas Conv1D (128, 256, 512 filtros) + BatchNorm + Dropout.

| Capa                  | Hiperparámetros                     | Justificación Técnica                  |
|-----------------------|------------------------------------|----------------------------------------|
| **InputLayer**        | `(None, 4000, 1)`                 | 4 seg @ 1kHz (óptimo para sonidos urbanos) |
| **Conv1D + BatchNorm**| 128 filtros, kernel=5, strides=2   | Gran kernel para patrones de larga duración |
| **Conv1D + BatchNorm**| 256 filtros, kernel=3, dilation=2  | Captura dependencias multi-escala       |
| **DepthwiseConv1D**   | 512 filtros, kernel=3              | Extracción eficiente de features        |
| **GlobalAvgPooling1D**| -                                  | Reduce parámetros vs Flatten            |

*   GlobalAveragePooling para reducir overfitting.

Entrenamiento:

*   Optimizador: Adam (LR=0.0001).

*   Early Stopping (paciencia=10 épocas).

SMOTE para balancear clases desequilibradas.

# Resultados

La siguiente tabla presenta el desempeño del modelo en cada una de las 10 clases de sonidos urbanos, evaluado mediante Precisión (Precision), Recall (Sensibilidad) y F1-Score, junto con el número de muestras de prueba para cada categoría:

| Clase               | Precisión | Recall | F1-Score | Muestras |
|---------------------|----------:|-------:|---------:|---------:|
| air_conditioner     |      0.91 |   0.91 |     0.91 |      200 |
| car_horn            |      0.81 |   0.85 |     0.83 |       86 |
| children_playing    |      0.85 |   0.78 |     0.81 |      200 |
| dog_bark            |      0.94 |   0.83 |     0.88 |      200 |
| drilling            |      0.90 |   0.86 |     0.88 |      200 |
| engine_idling       |      0.93 |   0.91 |     0.92 |      200 |
| gun_shot            |      0.86 |   0.97 |     0.91 |       75 |
| jackhammer          |      0.84 |   0.90 |     0.87 |      200 |
| siren               |      0.91 |   0.92 |     0.92 |      186 |
| street_music        |      0.77 |   0.84 |     0.80 |      200 |

*   Precisión (Precision): Indica cuántos de los ejemplos clasificados como positivos realmente son positivos. Ejemplo: Para gun_shot, un 86% de precisión significa que, de todos los audios que el modelo predijo como "disparo", el 86% eran correctos.

*   Recall (Sensibilidad): Mide la capacidad del modelo para detectar todos los casos positivos. Ejemplo: gun_shot tiene un 97% de recall, lo que indica que el modelo identifica casi todos los disparos reales, pero con algunos falsos positivos.

*   F1-Score: Combinación de precisión y recall (ideal para clases desbalanceadas). Ejemplo: car_horn (F1=0.83) tiene un desempeño bueno a pesar de tener solo 86 muestras, gracias al uso de SMOTE.

### Métricas globales

Accuracy: El modelo clasifica correctamente el 87% de los audios en total.

Weighted avg F1: Media del F1-Score de todas las clases ponderado es de 0.87.

### Gráfica de entrenamiento

![imagen 2](https://drive.google.com/uc?id=1cpcct2YAvN2T_Qd_NbnuAWaqpo972Gyt)

La gráfica muestra la evolución de las métricas de pérdida (loss) y exactitud (accuracy) durante el entrenamiento y validación del modelo a lo largo de las 100 épocas.

El modelo converge establemente después de ~60 épocas.

Sin overfitting (brecha mínima entre train/val).

# Conclusiones

Este proyecto ha demostrado de manera contundente que la sinergia entre técnicas avanzadas de procesamiento de señales (transformada Wavelet + MFCCs) y arquitecturas de Deep Learning (CNN 1D) ofrece un enfoque poderoso para la clasificación robusta de sonidos urbanos, superando significativamente los métodos tradicionales basados únicamente en características acústicas convencionales.

###Contribuciones Clave
Alto Desempeño en Clasificación:

*   Se logró un accuracy del 87% en un dataset desbalanceado (UrbanSound8K), con un F1-score promedio de 0.87, demostrando que el modelo generaliza efectivamente incluso en entornos acústicos complejos.

*   Clases críticas como gun_shot (recall: 97%) y siren (F1: 0.92) fueron identificadas con alta confiabilidad, lo que es crucial para aplicaciones de seguridad pública.

Solución a Desafíos Comunes:

*   El uso de SMOTE permitió mejorar el F1-score en clases minoritarias (ej: car_horn).

*   La transformada Wavelet complementó a los MFCCs para capturar tanto detalles espectrales como temporales, mejorando la discriminación de sonidos con firmas acústicas similares (ej: drilling vs jackhammer).

Modelo Robustez y Eficiencia:

*   La arquitectura CNN 1D, combinada con Batch Normalization y Dropout, mostró un entrenamiento estable sin overfitting, como lo confirman las curvas de aprendizaje.

*   A pesar de la complejidad añadida por la extracción de características wavelet, el modelo mantuvo un tiempo de inferencia aceptable (~2 segundos en CPU), con margen para optimización.

###Limitaciones y Lecciones Aprendidas
*   Street Music (F1: 0.80) sigue siendo un reto debido a su diversidad acústica, sugiriendo la necesidad de técnicas de aumento de datos más sofisticadas (ej: mezcla de espectrogramas).

*   La dependencia de múltiples etapas de preprocesamiento (wavelet + MFCCs) incrementa la latencia, lo que podría ser crítico para aplicaciones en tiempo real.